In [ ]:
import pandas as pd
import zipfile
from zipfile import ZipFile 

train = pd.read_csv("/kaggle/input/playground-series-s4e7/train.csv")
test = pd.read_csv("/kaggle/input/playground-series-s4e7/test.csv")

In [ ]:
train.info()

In [ ]:
# drop id
train = train.drop('id', axis = 1)
test_id = test.id
test = test.drop('id', axis = 1)

In [ ]:

# convert obj to cat
categorical = ["Region_Code", "Policy_Sales_Channel", "Gender", "Vehicle_Damage", "Vehicle_Age"]
test[["Region_Code", "Policy_Sales_Channel"]] = test[["Region_Code", "Policy_Sales_Channel"]].astype('int32')
test[categorical] = test[categorical].astype("category")

In [ ]:
# convert obj to cat
categorical = ["Region_Code", "Policy_Sales_Channel", "Gender", "Vehicle_Damage", "Vehicle_Age"]
train[["Region_Code", "Policy_Sales_Channel"]] = train[["Region_Code", "Policy_Sales_Channel"]].astype('int32')
train[categorical] = train[categorical].astype("category")

In [ ]:
# convert numerics into int 32 

import numpy as np
# integer
d = dict.fromkeys(train.select_dtypes(np.int64).columns, np.int32)
train = train.astype(d)

# float
d2 = dict.fromkeys(train.select_dtypes(np.float64).columns, np.int32)
train = train.astype(d2)

In [ ]:
# convert numerics into int 32 

import numpy as np
# integer
d = dict.fromkeys(test.select_dtypes(np.int64).columns, np.int32)
test = test.astype(d)

# float
d2 = dict.fromkeys(test.select_dtypes(np.float64).columns, np.int32)
test = test.astype(d2)

In [ ]:
from sklearn.model_selection import train_test_split
X = train.drop("Response", axis = 1)
y = train["Response"]


X_train, X_val, y_train, y_val = train_test_split(X,y,
                                                    test_size = 0.2,
                                                    random_state = 42,
                                                    stratify = y)

In [ ]:
from sklearn.metrics import f1_score, recall_score, confusion_matrix,roc_auc_score
import xgboost as xgb

xgtrain = xgb.DMatrix(X_train,y_train, enable_categorical= True)
xgtest  = xgb.DMatrix(X_val,y_val, enable_categorical= True)
params_1 = {"objective": "binary:logistic"}

n = 1000

results = xgb.cv(params_1,
                 xgtrain,
                 num_boost_round = n,
                 nfold=5,
                 metrics = ["logloss","auc","error"],
                 early_stopping_rounds=20
                 )


In [ ]:
clf_1 = xgb.XGBClassifier(**params_1)

clf_1.fit(X_train,y_train)

pred_1 = clf_1.predict(X_val)

print(f"f1 score : {f1_score(y_val, pred_1)}\n")
print(f"confusion Matrix:\n{confusion_matrix(y_val, pred_1)}\n")